In [5]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series  # for convenience
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import curve_fit
from scipy import stats  
from matplotlib import cm
import math
from scipy.misc import imsave
from scipy.misc import imread
import re
import glob, os
shot = 232
picL = 18333
picR = 18158
par1 = 4946
par2 = 3384

v0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL_sum.tif'.format(shot,shot,picL))
v0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR_sum.tif'.format(shot,shot,picR))
bk0L = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameL0.tif'.format(shot,shot,picL))
bk0R = imread('./Data/Shot{}/Clean_Data_Shot{}_Cam_{}/FrameR0.tif'.format(shot,shot,picR))

Freq = 100.*1000.
DeltaT = 1/Freq
def Parabola(x,a,b,c):
    y = a+b*x+c*x**2
    return y
#fig = plt.figure(figsize=[12,12])
#ax = fig.gca(projection='3d')
FitList = []
#list = pd.read_csv('./Data/Shot{}/3D/list.csv'.format(shot),header=None)
#list.columns = ['TrackL','TrackR']
#list.columns = ['TrackL']
#for it in range(len(list['TrackL'])):
#    itL = list['TrackL'].iloc[it]
    #itR = list['TrackR'].iloc[it]
    #file = "./Data/Shot{}/3D/3DErrEpilineCheckPinghanShot{}Pair{}_{}.csv".format(shot,shot,itL,itR)
#    file = "./Data/Shot{}/3D/3DErrEpilineCheckPinghanShot{}Pair{}.csv".format(shot,shot,itL)
 
for counter, file in enumerate(glob.glob("./Data/Shot{}/3D/3DErrEpilineCheckPinghanShot{}Pair*.csv".format(shot,shot))):    
    df = pd.read_csv(file)
    f1 = file.split('/')
    f2 = f1[4].split('.csv') 
    f3 = f2[0].split("3DErrEpilineCheckPinghanShot{}Pair".format(shot))
    index = f3[1]
    f4 = index.split('_')
    Par1 = int(f4[0])
    Par2 = int(f4[1])
    if( Par1 == par1 and Par2 == par2):
        print(index,Par1,Par2)    
    
        Track2D = pd.read_csv('./Data/Shot{}/TrackPair/Pair_{}.csv'.format(shot,index))
        Track2D = Track2D[:-1]
        #Track3D = pd.concat([df, Track2D['Frame']], axis=1)
        #Track3D['Time']=Track3D['Frame']*DeltaT
        Track2D['InvFrame']=6773-Track2D['Frame']
        Track3D = pd.concat([df, Track2D['InvFrame']], axis=1)
        Track3D['Time']=Track3D['InvFrame']*DeltaT
        XL=Track3D['Left xx']
        YL=Track3D['Left yy']
        XR=Track3D['Right xx']
        YR=Track3D['Right yy']
        X=Track3D['3Dxx']
        Y=Track3D['3Dyy']
        Z=Track3D['3Dzz']
        T=Track3D['Time']   
        params_x, pcov_x = curve_fit(Parabola, T, X)
        params_y, pcov_y = curve_fit(Parabola, T, Y)
        params_z, pcov_z = curve_fit(Parabola, T, Z)
        para_list = np.concatenate((params_x,params_y,params_z), axis=0)
        #para_list = [count,params_x[0],params_x[1],params_x[2],params_y[0],params_y[1],params_y[2],params_z[0],params_z[1],params_z[2]]
    #print(para_list)
        #FitList.append(para_list)
    
        fit_X = Parabola(T, params_x[0], params_x[1],params_x[2])
        fit_Y = Parabola(T, params_y[0], params_y[1],params_y[2])
        fit_Z = Parabola(T, params_z[0], params_z[1],params_z[2])
    
        fig = plt.figure(figsize = (12,12))  # make a new figure
        #f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2) 
        ax1 = fig.add_subplot(2, 2, 1)
        ax1.scatter(T,X,color='r',marker=".",s=1)
        ax1.plot(T,fit_X)
        ax1.set_xlabel('T (s)', fontsize=30,labelpad=15)
        ax1.set_ylabel('X (m)', fontsize=40,labelpad=20)
        ax2 = fig.add_subplot(2, 2, 2)
        ax2.scatter(T,Y,color='r',marker=".",s=1)
        ax2.plot(T,fit_Y)
        ax2.set_xlabel('T (s)', fontsize=30,labelpad=15)
        ax2.set_ylabel('Y (m)', fontsize=40,labelpad=20)
        ax3 = fig.add_subplot(2, 2, 3)
        ax3.scatter(T,Z,color='r',marker=".",s=1)
        ax3.plot(T,fit_Z)
        ax3.set_xlabel('T (s)', fontsize=30,labelpad=15)
        ax3.set_ylabel('Z (m)', fontsize=40,labelpad=20)
        ax4 = fig.add_subplot(2, 2, 4, projection='3d')
        ax4.view_init(10, -30)
        ax4.set_xlabel('X (m)', fontsize=40,labelpad=20)
        ax4.set_ylabel('Y (m)', fontsize=40,labelpad=20)
        ax4.set_zlabel('Z (m)', fontsize=40,labelpad=20)      
        ax4.plot(X, Y, Z)
        plt.savefig("3DFit_{}_{}.pdf".format(shot,index))

('4946_3384', 4946, 3384)
